In [65]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280648 entries, 0 to 280647
Data columns (total 44 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   fen                           280648 non-null  object 
 1   elo                           280648 non-null  float64
 2   move                          280648 non-null  object 
 3   best_pv                       280648 non-null  object 
 4   best_move                     280648 non-null  object 
 5   best_mate                     280648 non-null  float64
 6   best_pv2                      280648 non-null  object 
 7   best_move2                    280648 non-null  object 
 8   best_score2                   280648 non-null  float64
 9   correct                       280648 non-null  bool   
 10  fullmove_number               280648 non-null  int64  
 11  is_check                      280648 non-null  bool   
 12  our_number_of_bishop_moves    280648 non-nul

In [2]:
import utils
display(HTML('''<style>
    .widget-label { min-width: 200px !important; }
    .widget-slider { width: 500px !important; }
</style>'''))


def int_range_slider(df, column):
    min = df[column].min()
    max = df[column].max()
    
    if abs(min) > 1000 or abs(max) > 1000:
        step = 100
        min = int(min / 100) * 100
        max = int(max / 100) * 100
    
    return widgets.IntRangeSlider(
        value=[min, max],
        min=min,
        max=max,
        step=1,
        description=column,
        continuous_update=False
    )


def f(df, **kwargs):
    for column, value in kwargs.items():
        min, max = value
        df = df[df[column] >= min]
        df = df[df[column] <= max]
    
    html = utils.df_to_boards(df[:55])
    return HTML(html)


# columns = ['fullmove_number', 'our_number_of_moves', 'best_mate']
# non_feature_columns = ['fen', 'move', 'best_move', 'best_move2', 'best_pv', 'best_pv2']
# columns = list(set(df.columns) - set(non_feature_columns))

columns = [
    'best_mate',
    'best_pv_our_number_of_pieces_moved',

#     'our_number_of_moves', 
#     'their_number_of_moves',
#     'fullmove_number',
#     'our_number_of_captures',
#     'correct',
#     'elo',
    
    # TODO: distance moved.
    # TODO: their_hanging_pieces
    # 
    
]

# TODO: show user move if not best move (and score?)

controls = {}
for column in columns:
    controls[column] = int_range_slider(df, column)
interact(f, df=fixed(df), **controls);

KeyError: 'best_pv_our_number_of_pieces_moved'

# Plots

In [61]:
# TODO: use csv for this.

# TODO: combine this with boards?
# TODO: make this a single cell in a notebook that is our vis tool.
import plotly.express as px
from plotly.offline import iplot


# TODO: how to do filtering?
# TODO: what other graphing functionality do we want?
def f(x):
    fig = px.histogram(feature_df, x=x, color='correct')
    iplot(fig)


# range_slider = widgets.IntRangeSlider(
#     value=[5, 7],
#     min=0,
#     max=10,
# )


# TODO: how to get all features? Use get features func on classes?
interact(f, x=features.class_features(features.BoardFeatures));

NameError: name 'features' is not defined

In [68]:
import plotly.express as px

column = 'difficulty'

plot_df = df
plot_df['elo'] = df['elo'].apply(lambda x: int(x / 200) * 200)
plot_df = plot_df[plot_df['elo'] <= 2000]
plot_df = plot_df[plot_df['elo'] >= 800]


# plot_df = plot_df[plot_df['best_mate'] < 6]
plot_df = plot_df[plot_df['fullmove_number'] <= 50]
# plot_df['fullmove_number'] = df['fullmove_number'].apply(lambda x: int(x / 10) * 10)
plot_df['difficulty'] = df['difficulty'].apply(lambda x: int(x * 5) * 20)
plot_df = plot_df.groupby([column, 'elo'], as_index=False).mean()


layout = {
    'title': '',
    'plot_bgcolor': 'white',
     
    'width': 600,
    'height': 400,

    'xaxis': {
        'title_text': 'Elo',
        'color': 'grey',
        'linecolor': 'grey',
        'ticks': 'outside',
        'tickwidth': 1,
        'tickcolor': 'grey',
        'ticklen': 5,
        'showgrid': False,
    },

    'yaxis': {
        'title_text': 'Accuracy',
        'range': [0, 1],
        'tickformat': ',.0%',
        'color': 'grey',
        'linecolor': 'grey',
        'ticks': 'outside',
        'tickwidth': 1,
        'tickcolor': 'grey',
        'ticklen': 5,
        'showgrid': False,
    }
 }

colors = {
    "0": "#c6ecff",
    "5": "#aee2ff",
    "10": "#96d5ff",
    "20": "#69b6fb",
    "30": "#4095f2",
    "40": "#1a73e8",
    "50": "#206080"
}

fig = px.line(plot_df, x='elo', y='correct', color=column, color_discrete_sequence=list(colors.values()))
fig.update_layout(layout)

In [56]:
df

,fen,elo,move,best_pv,best_move,best_mate,best_pv2,best_move2,best_score2,correct,...,our_queens,our_rooks,piece_count,their_bishops,their_knights,their_pawns,their_piece_count,their_queens,their_rooks,difficulty
0,r5k1/p3rpp1/2Bb3p/P6q/4p3/P1P3Pb/3PNK2/RNBQ3R ...,1346.0,h5f3,"['h5f3', 'f2e1', 'f3h1', 'e2g1', 'h1g1', 'e1e2...",h5f3,4.0,"['h5f5', 'f2e1', 'a8d8', 'd1a4', 'h3g4', 'a4c2']",h5f5,-161.0,True,...,1,2,24,2,2,5,13,1,2,0.555605
1,r5k1/p3rpp1/2Bb3p/P7/4p3/P1P2qPb/3PN3/RNBQK2R ...,1346.0,f3h1,"['f3h1', 'e2g1', 'h1g1', 'e1e2', 'h3g4']",f3h1,3.0,"['d6c5', 'e2d4', 'f3h1', 'e1f2', 'h1h2', 'f2e3...",d6c5,1590.0,True,...,1,2,24,2,2,5,13,1,2,0.529898
2,r1b2rk1/pp3pp1/3qp3/7Q/3p4/5N2/PPP2PPP/R5K1 w ...,2074.0,f3g5,"['f3g5', 'f8d8', 'h5f7', 'g8h8', 'f7h5', 'h8g8...",f3g5,7.0,"['a1d1', 'd6f4', 'h2h3', 'f4f5', 'h5g4', 'e6e5...",a1d1,-576.0,True,...,1,1,21,1,0,6,11,1,2,0.791094
3,r1b1r1k1/pp3pp1/3qp3/6NQ/3p4/8/PPP2PPP/R5K1 w ...,2074.0,h5f7,"['h5f7', 'g8h8', 'f7e8', 'd6f8', 'e8f8']",h5f7,3.0,"['g2g3', 'd6c7', 'h5h7', 'g8f8', 'h7h8', 'f8e7...",g2g3,-201.0,True,...,1,1,21,1,0,6,11,1,2,0.875332
4,k2r1b2/p2Np1p1/4P3/3p2Q1/3PnB1p/2P4K/PPq5/R3R3...,1655.0,g5d5,['g5d5'],g5d5,1.0,"['g5e5', 'e4d6', 'e5d5', 'd6b7', 'a1c1', 'c2h7...",g5e5,943.0,True,...,1,2,21,1,1,5,10,1,1,0.706450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53044,rn2r1k1/p5pp/2p5/3q1p2/1Q1n1B2/1P4P1/P4P1P/2R3...,1404.0,d4e2,"['d4e2', 'g1f1', 'd5h1']",d4e2,2.0,"['g8h8', 'f2f3', 'd4f3', 'g1f2', 'f3d4', 'f4e3']",g8h8,1605.0,True,...,1,2,20,1,0,5,9,1,1,0.602676
53045,4R1rk/p1p3pp/2p5/8/8/8/PP4PP/3q1Q1K b - - 4 28,1463.0,d1f1,['d1f1'],d1f1,1.0,"['d1d2', 'e8e2', 'd2d5', 'e2e1', 'd5a2', 'f1f4...",d1d2,123.0,True,...,1,1,15,0,0,4,7,1,1,0.807846
53046,r1b2rk1/ppp2p1p/2n1pp2/q1Pp4/3P4/2PB1Q2/PN1B1P...,1552.0,f3g4,"['d2h6', 'c6e5', 'f3f6', 'e5f3', 'g2f3', 'c7c6...",d2h6,4.0,"['f3h5', 'f6f5', 'd2h6', 'g8h8', 'h5g5']",f3h5,1373.0,False,...,1,2,28,1,1,8,14,1,2,0.584575
53047,r1b2r1k/ppp2p1p/2n1pp2/q1Pp4/3P2Q1/2PB4/PN1B1P...,1552.0,g4h5,"['g4h4', 'f6f5', 'h4f6', 'h8g8', 'd2h6', 'f5f4...",g4h4,4.0,"['d2h6', 'f8g8', 'g4h4', 'g8g6', 'd3g6', 'h8g8...",d2h6,2122.0,False,...,1,2,28,1,1,8,14,1,2,0.580778


In [69]:
df

,fen,elo,move,best_pv,best_move,best_mate,best_pv2,best_move2,best_score2,correct,...,our_queens,our_rooks,piece_count,their_bishops,their_knights,their_pawns,their_piece_count,their_queens,their_rooks,difficulty
0,r5k1/p3rpp1/2Bb3p/P6q/4p3/P1P3Pb/3PNK2/RNBQ3R ...,1200,h5f3,"['h5f3', 'f2e1', 'f3h1', 'e2g1', 'h1g1', 'e1e2...",h5f3,4.0,"['h5f5', 'f2e1', 'a8d8', 'd1a4', 'h3g4', 'a4c2']",h5f5,-161.0,True,...,1,2,24,2,2,5,13,1,2,0.436176
1,r5k1/p3rpp1/2Bb3p/P7/4p3/P1P2qPb/3PN3/RNBQK2R ...,1200,f3h1,"['f3h1', 'e2g1', 'h1g1', 'e1e2', 'h3g4']",f3h1,3.0,"['d6c5', 'e2d4', 'f3h1', 'e1f2', 'h1h2', 'f2e3...",d6c5,1590.0,True,...,1,2,24,2,2,5,13,1,2,0.460652
2,r1b2rk1/pp3pp1/3qp3/7Q/3p4/5N2/PPP2PPP/R5K1 w ...,2000,f3g5,"['f3g5', 'f8d8', 'h5f7', 'g8h8', 'f7h5', 'h8g8...",f3g5,7.0,"['a1d1', 'd6f4', 'h2h3', 'f4f5', 'h5g4', 'e6e5...",a1d1,-576.0,True,...,1,1,21,1,0,6,11,1,2,0.223405
3,r1b1r1k1/pp3pp1/3qp3/6NQ/3p4/8/PPP2PPP/R5K1 w ...,2000,h5f7,"['h5f7', 'g8h8', 'f7e8', 'd6f8', 'e8f8']",h5f7,3.0,"['g2g3', 'd6c7', 'h5h7', 'g8f8', 'h7h8', 'f8e7...",g2g3,-201.0,True,...,1,1,21,1,0,6,11,1,2,0.132303
4,k2r1b2/p2Np1p1/4P3/3p2Q1/3PnB1p/2P4K/PPq5/R3R3...,1600,g5d5,['g5d5'],g5d5,1.0,"['g5e5', 'e4d6', 'e5d5', 'd6b7', 'a1c1', 'c2h7...",g5e5,943.0,True,...,1,2,21,1,1,5,10,1,1,0.284866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280643,rn2Q3/pp5k/2p1Nrp1/4Bp2/6bq/2NB4/PPP5/R4RK1 w ...,1800,e8e7,"['e8e7', 'f6f7', 'e7f7', 'h7h6', 'f7g7', 'h6h5...",e8e7,4.0,"['c3d1', 'h4h6', 'e5f6', 'h6e3', 'd1e3', 'b8d7...",c3d1,3330.0,True,...,1,2,22,1,1,5,11,1,2,0.622684
280644,1r3rk1/4p2p/5ppb/3b4/Pn6/4qPP1/3NB2P/R1Q1K2R b...,1400,b4d3,"['b4d3', 'e1d1', 'd3c1', 'f3f4', 'e3e2', 'd1c1...",b4d3,5.0,"['b8c8', 'e1f1', 'c8c1', 'a1c1', 'e3d2', 'c1d1...",b8c8,1707.0,True,...,1,2,21,1,1,4,10,1,2,0.562099
280645,3r1r1k/1p6/6B1/p1p1N3/2P2q2/2Qb4/P4PPP/4RRK1 w...,1400,e5d3,"['e5f7', 'h8g8', 'c3h8']",e5f7,2.0,"['g6d3', 'h8g8', 'c3a5', 'f4f6', 'e1e3', 'f8e8...",g6d3,1341.0,False,...,1,2,19,1,0,3,8,1,2,0.389994
280646,6k1/5pbp/4p1p1/2p1P3/8/4P1BP/b5PK/1q6 b - - 1 40,1800,a2d5,"['a2d5', 'g3f4', 'b1e4', 'h2g3', 'e4g2', 'g3h4...",a2d5,7.0,"['b1e4', 'g3h4', 'e4h4', 'g2g3', 'g7e5', 'h2h1...",b1e4,7170.0,True,...,1,0,15,1,0,4,6,0,0,0.613490


In [74]:
temp_df = df[:100]
temp_df['svg'] = temp_df.apply(lambda row: utils.fen_to_svg(row['fen'], row['best_move']), axis=1)

<ipython-input-74-1eb1b48cf833>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [88]:
HTML(temp_df[:3][['svg', 'best_mate', 'piece_count', 'fullmove_number', 'our_number_of_moves', 'their_number_of_moves', 'is_check', 'material_advantage', 'turn']].to_html(escape=False, index=False))

In [85]:
temp_df

,fen,elo,move,best_pv,best_move,best_mate,best_pv2,best_move2,best_score2,correct,...,our_rooks,piece_count,their_bishops,their_knights,their_pawns,their_piece_count,their_queens,their_rooks,difficulty,svg
0,r5k1/p3rpp1/2Bb3p/P6q/4p3/P1P3Pb/3PNK2/RNBQ3R ...,1200,h5f3,"['h5f3', 'f2e1', 'f3h1', 'e2g1', 'h1g1', 'e1e2...",h5f3,4.0,"['h5f5', 'f2e1', 'a8d8', 'd1a4', 'h3g4', 'a4c2']",h5f5,-161.0,True,...,2,24,2,2,5,13,1,2,0.436176,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
1,r5k1/p3rpp1/2Bb3p/P7/4p3/P1P2qPb/3PN3/RNBQK2R ...,1200,f3h1,"['f3h1', 'e2g1', 'h1g1', 'e1e2', 'h3g4']",f3h1,3.0,"['d6c5', 'e2d4', 'f3h1', 'e1f2', 'h1h2', 'f2e3...",d6c5,1590.0,True,...,2,24,2,2,5,13,1,2,0.460652,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
2,r1b2rk1/pp3pp1/3qp3/7Q/3p4/5N2/PPP2PPP/R5K1 w ...,2000,f3g5,"['f3g5', 'f8d8', 'h5f7', 'g8h8', 'f7h5', 'h8g8...",f3g5,7.0,"['a1d1', 'd6f4', 'h2h3', 'f4f5', 'h5g4', 'e6e5...",a1d1,-576.0,True,...,1,21,1,0,6,11,1,2,0.223405,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
3,r1b1r1k1/pp3pp1/3qp3/6NQ/3p4/8/PPP2PPP/R5K1 w ...,2000,h5f7,"['h5f7', 'g8h8', 'f7e8', 'd6f8', 'e8f8']",h5f7,3.0,"['g2g3', 'd6c7', 'h5h7', 'g8f8', 'h7h8', 'f8e7...",g2g3,-201.0,True,...,1,21,1,0,6,11,1,2,0.132303,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
4,k2r1b2/p2Np1p1/4P3/3p2Q1/3PnB1p/2P4K/PPq5/R3R3...,1600,g5d5,['g5d5'],g5d5,1.0,"['g5e5', 'e4d6', 'e5d5', 'd6b7', 'a1c1', 'c2h7...",g5e5,943.0,True,...,2,21,1,1,5,10,1,1,0.284866,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1k4r1/pppQ4/2np4/8/N2PPp2/2P5/PP3b1P/R1B4K b -...,1200,g8g1,['g8g1'],g8g1,1.0,"['c6e5', 'c1f4', 'e5d7', 'c3c4', 'f2d4', 'a4c3...",c6e5,177.0,True,...,1,20,1,1,6,11,1,1,0.238941,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
96,r1b1k2r/ppp2pp1/4p2p/3P4/3Nn2q/P7/2P2PPP/R1B1K...,1600,e4f2,"['h4f2', 'e1d1', 'e4c3']",h4f2,2.0,"['e6e5', 'c1e3', 'e5d4', 'e3d4', 'e8g8', 'a1c1...",e6e5,1389.0,False,...,2,25,2,1,6,12,0,2,0.237252,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
97,1r5k/R6p/5bpP/1p1b4/8/PP6/K2R4/5r2 b - - 0 37,1400,f1a1,['f1a1'],f1a1,1.0,"['b8e8', 'd2b2', 'e8e1', 'a7h7', 'h8h7']",b8e8,2183.0,True,...,2,14,0,0,3,6,0,2,0.463508,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."
98,r1bqkr2/ppppb1Qp/8/3P1n2/8/8/PPP1BPPP/RNB1K2R ...,1600,e2h5,"['e2h5', 'f8f7', 'h5f7']",e2h5,2.0,"['g7h7', 'e7b4', 'c2c3', 'd8e7', 'h7h5', 'f8f7...",g7h7,796.0,True,...,2,26,2,1,5,12,1,2,0.260146,"<svg xmlns=""http://www.w3.org/2000/svg"" versio..."


In [86]:
temp_df.columns

Index(['fen', 'elo', 'move', 'best_pv', 'best_move', 'best_mate', 'best_pv2',
       'best_move2', 'best_score2', 'correct', 'fullmove_number', 'is_check',
       'our_number_of_bishop_moves', 'our_number_of_captures',
       'our_number_of_checks', 'our_number_of_knight_moves',
       'our_number_of_moves', 'our_number_of_pawn_moves',
       'our_number_of_queen_moves', 'our_number_of_rook_moves',
       'their_number_of_bishop_moves', 'their_number_of_captures',
       'their_number_of_checks', 'their_number_of_knight_moves',
       'their_number_of_moves', 'their_number_of_pawn_moves',
       'their_number_of_queen_moves', 'their_number_of_rook_moves', 'turn',
       'material_advantage', 'our_bishops', 'our_knights', 'our_pawns',
       'our_piece_count', 'our_queens', 'our_rooks', 'piece_count',
       'their_bishops', 'their_knights', 'their_pawns', 'their_piece_count',
       'their_queens', 'their_rooks', 'difficulty', 'svg'],
      dtype='object')